# The Viterbi Algorithm

Imagine you’re playing a guessing game: your friend hides a toy and makes a series of noises—“meow,” “woof,” “chirp,” and so on. Your job is to figure out, at each sound, which toy they’re thinking of (cat, dog, bird, etc.).

A naïve approach would be to consider **every possible sequence** of guesses for the entire string of sounds—and then pick the overall best one. If there are many possible toys (hidden states) and your friend makes a long series of noises, that brute-force search becomes exponentially expensive.

The Viterbi algorithm slashes this down via **dynamic programming**:

---

## 1. States and Observations

- **Hidden states**  
  Think of each toy (cat, dog, bird, etc.) as a hidden state.

- **Observations**  
  The input sequence is the list of sounds your friend makes.

---

## 2. DP Table Definition

Define a table `V[t][s]` that stores, for each time step `t` and each state `s`, the highest log-probability of any path ending in state `s` after observing the first `t` sounds. In other words, it’s the best score you can get up to time `t` if you assume the hidden state at `t` is `s`.

---

## 3. Recurrence Relation

At each new time step `t` and for each state `s`:

1. Look at every possible previous state `r`.
2. For each, add:
   - The best score at time `t-1` for state `r` (`V[t-1][r]`).
   - The log-probability of transitioning from `r` to `s`.
3. Take the maximum of those sums.
4. Add the log-probability of emitting the observed sound at time `t` from state `s`.
5. Store both the maximum score in `V[t][s]` and the corresponding previous state `r` in a backpointer table `BP[t][s]` so you can reconstruct the path later.

---

## 4. Initialization & Termination

### Initialization (time step 0)

- For each state `s`, compute the score of starting in `s` and emitting the first sound. Store it in `V[0][s]`.  
- Set `BP[0][s] = Start` for all `s`.

### Termination (final time step)

- Among all states, pick the one with the highest score at the final time step (including the transition to an “End” state, if your model has one).
- Backtrack from that state using `BP` to recover the entire best path.

---

## 5. Time Complexity

- **Brute-force**: Exponential in sequence length (checking every state sequence).
- **Viterbi**: For each time step, you examine all pairs of current and previous states—so it runs in polynomial time (quadratic in the number of states, linear in sequence length).

---

By reusing previously computed scores instead of exploring every possible sequence, the Viterbi algorithm turns an intractable search into an efficient dynamic-programming solution—an enormous practical improvement.


In [12]:
import math
from typing import List, Dict, Tuple

LogProb = float
State = str
Symbol = str

class HMM:
    """
    A simple three-state Hidden Markov Model for DNA sequences:
      - Exon (E)
      - FivePrime (5)
      - Intron (I)
    All probabilities are stored in log-space.
    """
    
    def __init__(self):
        # Define hidden states
        self.states: List[State] = ['E', '5', 'I']
        
        # Transition log-probabilities: from each state (inc. Start) to each other (inc. End)
        self.log_trans: Dict[State, Dict[State, LogProb]] = {
            'Start': {'E': 0.0,       '5': -math.inf, 'I': -math.inf, 'End': -math.inf},
            'E':     {'E': math.log(0.9), '5': math.log(0.1), 'I': -math.inf,    'End': -math.inf},
            '5':     {'E': -math.inf,    '5': -math.inf,    'I':  math.log(1.0), 'End': -math.inf},
            'I':     {'E': -math.inf,    '5': -math.inf,    'I': math.log(0.9), 'End': math.log(0.1)},
        }
        
        # Emission log-probabilities: from each hidden state to each DNA base
        self.log_emit: Dict[State, Dict[Symbol, LogProb]] = {
            'E': {'A': -math.log(4),  'C': -math.log(4),  'G': -math.log(4),  'T': -math.log(4)},
            '5': {'A': math.log(0.05),'C': -math.inf,     'G': math.log(0.95),'T': -math.inf},
            'I': {'A': math.log(0.4), 'C': math.log(0.1), 'G': math.log(0.1), 'T': math.log(0.4)},
        }

    def score_path(self, path: str, sequence: str) -> LogProb:
        """
        Compute the total log-probability of observing 'sequence'
        while passing through the fixed 'path' of hidden states.
        """
        total = 0.0
        
        # Initial transition: Start -> first state
        first = path[0]
        total += self.log_trans['Start'][first]
        total += self.log_emit[first][sequence[0]]
        
        # Transitions + emissions along the path
        for prev_state, curr_state, obs in zip(path, path[1:], sequence[1:]):
            total += self.log_trans[prev_state][curr_state]
            total += self.log_emit[curr_state][obs]
        
        # If ending in an intron, include its End transition
        if path[-1] == 'I':
            total += self.log_trans['I']['End']
        
        return total

    def most_likely_path(self, sequence: str) -> str:
        """
        Use the Viterbi algorithm to find the highest-probability
        state path for the given observation sequence.
        """
        n = len(sequence)
        V: List[Dict[State, LogProb]] = [{} for _ in range(n)]
        backptr: List[Dict[State, State]] = [{} for _ in range(n)]
        
        # Initialization (t = 0)
        for s in self.states:
            V[0][s] = self.log_trans['Start'][s] + self.log_emit[s][sequence[0]]
            backptr[0][s] = 'Start'
        
        # Recursion (t = 1 ... n-1)
        for t in range(1, n):
            for curr in self.states:
                best_score, best_prev = -math.inf, None
                for prev in self.states:
                    score = V[t-1][prev] + self.log_trans[prev][curr] + self.log_emit[curr][sequence[t]]
                    if score > best_score:
                        best_score, best_prev = score, prev
                V[t][curr] = best_score
                backptr[t][curr] = best_prev  # store the argmax
        
        # Termination: choose best final state including transition to 'End'
        final_scores: Dict[State, LogProb] = {
            s: V[n-1][s] + (self.log_trans[s]['End'] if s == 'I' else -math.inf)
            for s in self.states
        }
        last_state = max(final_scores, key=final_scores.get)
        
        # Backtrack to recover the full path
        path: List[State] = [last_state]
        for t in range(n-1, 0, -1):
            path.insert(0, backptr[t][path[0]])
        
        return ''.join(path)


# === Example usage ===
if __name__ == "__main__":
    model = HMM()
    seq = "CTTCATGTGAAAGCAGACGTAAGTCA"
    
    # Score a fixed path
    test_path = "EEEEEEEEEEEEEEEEEE5IIIIIII"
    print("Log-prob of path:", model.score_path(test_path, seq))
    
    # Find the Viterbi path
    best = model.most_likely_path(seq)
    print("Viterbi best path:", best)


Log-prob of path: -41.21967768602254
Viterbi best path: EEEEEEEEEEEEEEEEEE5IIIIIII
